# Platform Testing

# Imports

In [1]:
import subprocess
import time
import yaml
import sys

# Prepare paths and load yaml config file

In [2]:
# Prepare the paths that will be used during the test
if "ipykernel_launcher.py" in sys.argv[0]:
    yamlInfoFile="TestSetup.yaml"
else:
    if len(sys.argv) > 1:
        yamlInfoFile=sys.argv[1]
    else:
        yamlInfoFile="TestSetup.yaml"
    
print("Using setup file: {}".format(yamlInfoFile))
fileExt = ""
commandsFolder = ""
startCommand = ""
ProcessTail = ""
LogCommandTail = ""

with open(yamlInfoFile) as file:
    doc = yaml.load(file, Loader=yaml.FullLoader)

    for item, doc in doc.items():
        if item == "Setup":
            Setup = doc
        elif item == "Examples":
            Examples = doc    
        elif item == "Deployments":
            Deployments = doc    
        else:
            Machines = doc

if Setup['TestMgrOS'] == "Linux":
    fileExt = ".sh"
    commandsFolder = "Commands/Linux/"
    startCommand = "setsid"
    ScriptServerPath = "Scripts/Server/"
    ScriptDevicePath = "Scripts/Device/"
    DeviceFolderPath = "DeviceFolder/"
    ProcessTitle = ""
    LogsPath = "Logs/"
    SleepTime3Sec = "sleep 3\n"
    ExamplesPath = "examples/"
    AddScriptsServerPath = "Scripts/Server/"
    AddScriptsDevicePath = "Scripts/Device/"
    for VM in Machines:
        VM['Password'] = "\'{}\'".format(VM['Password'])
elif Setup['TestMgrOS'] == "Win":
    fileExt = ".bat"
    commandsFolder = "Commands\\Win\\"
    startCommand = "start"
    ScriptServerPath = "Scripts\\Server\\*"
    ScriptDevicePath = "Scripts\\Device\\*"
    DeviceFolderPath = "DeviceFolder\*"
    ProcessTitle = "\"Running\""
    LogsPath = "Logs\\"
    SleepTime3Sec = "TIMEOUT 3 > nul\n"
    ExamplesPath = "examples\\*"
    AddScriptsServerPath = "Scripts\\Server\\"
    AddScriptsDevicePath = "Scripts\\Device\\"
    
batMoveFiles = "{}MoveFiles{}".format(commandsFolder, fileExt)
batIIPPrerequisiteInstallation = "{}IIPPrerequisiteInstallation{}".format(commandsFolder, fileExt)
batCleanAll = "{}CleanAll{}".format(commandsFolder, fileExt)
batDockerInstallation = "{}DockerInstallation{}".format(commandsFolder, fileExt)
batInstallPlatform = "{}InstallPlatform{}".format(commandsFolder, fileExt)
batRunDockerRegistry = "{}RunDockerRegistry{}".format(commandsFolder, fileExt)
batStopDockerRegistry = "{}StopDockerRegistry{}".format(commandsFolder, fileExt)
batPullRunEcsContainer = "{}PullRunEcsContainer{}".format(commandsFolder, fileExt)
batStopEcsContainer = "{}StopEcsContainer{}".format(commandsFolder, fileExt)
batStartPlatform = "{}StartPlatform{}".format(commandsFolder, fileExt)
batJoinPlatform = "{}JoinPlatform{}".format(commandsFolder, fileExt)
batCliContainersCommand = "{}CliContainersCommand{}".format(commandsFolder, fileExt)
batCliServicesCommand = "{}CliServicesCommand{}".format(commandsFolder, fileExt)
batCliDeployCommand = "{}CliDeployCommand{}".format(commandsFolder, fileExt)
batLeavePlatform = "{}LeavePlatform{}".format(commandsFolder, fileExt)
batStopPlatform = "{}StopPlatform{}".format(commandsFolder, fileExt)
batRebootVMs = "{}RebootVMs{}".format(commandsFolder, fileExt)
batGetCompleteAlert = "{}GetCompleteAlert{}".format(commandsFolder, fileExt)
batCheckDevices = "{}CheckDevices{}".format(commandsFolder, fileExt)
executionCompletedFile = "CompletedAlert.info"

print("Setup file is Loaded")

Using setup file: TestSetup.yaml
Setup file is Loaded


# Prepare Dynamic Scripts 

In [3]:
# Prepare variables and clean or create empty the bat files
RegistryList = []
VMHomePath = ""
VMHomeFilesPath = ""
VMHomeExamplesPath = ""
VMFileExt = ""
moveFilesLine = ""
compileExamplesLine = ""
IIPPrerequisiteInstallationLine = ""
installPlatformLine = ""
runDockerRegistryLine = ""
stopDockerRegistryLine = ""
pullRunEcsContainerLine = ""
stopEcsContainerLine = ""
startPlatformLine = ""
joinPlatformLine = ""
cliContainersCommandLine = ""
LeavePlatformLine = ""
stopPlatformLine = ""
rebootVMLine = ""
checkDevicesLines = ""
cleanAllLines = ""

writeFile = open(batMoveFiles, "w")
writeFile.write(moveFilesLine)
writeFile.close()
writeFile = open(batIIPPrerequisiteInstallation, "w")
writeFile.write(IIPPrerequisiteInstallationLine)
writeFile.close()
writeFile = open(batInstallPlatform, "w")
writeFile.write(installPlatformLine)
writeFile.close()
writeFile = open(batRunDockerRegistry, "w")
writeFile.write(runDockerRegistryLine)
writeFile.close()
writeFile = open(batStopDockerRegistry, "w")
writeFile.write(stopDockerRegistryLine)
writeFile.close()
writeFile = open(batPullRunEcsContainer, "w")
writeFile.write(pullRunEcsContainerLine)
writeFile.close()
writeFile = open(batStopEcsContainer, "w")
writeFile.write(stopEcsContainerLine)
writeFile.close()
writeFile = open(batJoinPlatform, "w")
writeFile.write(joinPlatformLine)
writeFile.close()
writeFile = open(batCliContainersCommand, "w")
writeFile.write(cliContainersCommandLine)
writeFile.close()
writeFile = open(batStartPlatform, "w")
writeFile.write(startPlatformLine)
writeFile.close()
writeFile = open(batLeavePlatform, "w")
writeFile.write(LeavePlatformLine)
writeFile.close()
writeFile = open(batStopPlatform, "w")
writeFile.write(stopPlatformLine)
writeFile.close()
writeFile = open(batRebootVMs, "w")
writeFile.write(rebootVMLine)
writeFile.close()
writeFile = open(batCheckDevices, "w")
writeFile.write(checkDevicesLines)
writeFile.close()
writeFile = open(batCleanAll, "w")
writeFile.write(cleanAllLines)
writeFile.close()

for VM in Machines:
    
    # Skip the VMs with "Skip : True" in VMsSetup.yaml
    if not VM['Included']:
        continue
    if VM['OS'] == "Linux":
        VMHomeFilesPath = "Files/"
        VMFileExt = ".sh"
    elif VM['OS'] == "Win":
        VMHomeFilesPath = "Files\\\\"
        VMFileExt = ".bat"
          
    rebootVMLine="plink -t -batch -pw {} {}@{} {}RestartVM{} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"))
    moveFilesMkdirLine="plink -t -batch -pw {} {}@{} \"mkdir {}Files\"\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomePath)
    mkExecuteFilesLine="plink -t -batch -pw {} {}@{} \"chmod 755 {}Files/*{}\"\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomePath, VMFileExt)
    # Create the bat files for the Server
    if VM['Role'] == "Server":
        # Create bat file to move the auto scripts to the Server
        moveFilesLine="echo y | pscp -r -pw {} \"{}\" {}@{}:{}Files\n".format(VM['Password'], ScriptServerPath, VM['Username'], VM['IP'], VMHomePath)
        # Create bat file for install the platform as Server
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Install_{}.log &".format(LogsPath, VM['IP'])
        if VM['RunAsContainer']:
            installPlatformLine=""
        else:
            installPlatformLine="{} {} plink -t -batch -pw {} {}@{} {}IIPInstallationServer{} {} {} {} {} {} {} {} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['EcsSvcMgrContainers'], Setup['DockerRegistry'], Setup['IIPEasyTracing'], VM['Username'], Setup['ArtifactsFolder'], Setup['NewInstallFiles'], Setup['LocalBrokerPort'], ProcessTail)
        
        # Create bat file to start the platform Server
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Start_{}.log &".format(LogsPath, VM['IP'])
        startPlatformLine="{} {} plink -t -batch -pw {} {}@{} {}StartAllPlatformWithBroker{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
        batfile = open(batStartPlatform, "a")
        batfile.write(startPlatformLine)
        batfile.close()
        # Create bat file to stop the platform Server
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Stop_{}.log &".format(LogsPath, VM['IP'])
        stopPlatformLine="plink -t -batch -pw {} {}@{} {}StopAllPlatformWithBroker{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
        batfile = open(batStopPlatform, "a")
        batfile.write(stopPlatformLine)
        batfile.close()
    # Create the bat files for the Devices
    elif VM['Role'] == "Device":
        # Create bat file to move the auto scripts to the Device(s)
        moveFilesLine="echo y | pscp -r -pw {} \"{}\" {}@{}:{}Files\n".format(VM['Password'], ScriptDevicePath, VM['Username'], VM['IP'], VMHomePath)
        # Create bat file for install the platform as Device
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Install_{}.log &".format(LogsPath, VM['IP'])
        if VM['RunAsContainer']:
            installPlatformLine=""
            # Create bat file to pull and run Ecs container to join the platform
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Join_{}.log &".format(LogsPath, VM['IP'])
            pullRunEcsContainerLine="{} {} plink -t -batch -pw {} {}@{} {}PullRunEcsContainer{} {} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['DockerRegistry'], ProcessTail)
            writeFile = open(batJoinPlatform, "a")
            writeFile.write(pullRunEcsContainerLine)
            writeFile.close()
            # Create bat file to stop Ecs container on the Device(s)
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Leave_{}.log &".format(LogsPath, VM['IP'])
            stopEcsContainerLine="plink -t -batch -pw {} {}@{} {}StopEcsContainer{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
            writeFile = open(batLeavePlatform, "a")
            writeFile.write(stopEcsContainerLine)
            writeFile.close()
        else:
            # Create bat file to start and join ecs file to the platform
            installPlatformLine=""
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Join_{}.log &".format(LogsPath, VM['IP'])
            joinPlatformLine="{} {} plink -t -batch -pw {} {}@{} {}JoinPlatform{} {} {} {} {} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), VM['DeviceID'], VM['NewDeviceFiles'], Setup['MgtUiServerIP'], Setup['LocalBrokerPort'], ProcessTail)
            batfile = open(batJoinPlatform, "a")
            batfile.write(joinPlatformLine)
            batfile.close()
            # Create bat file to stop and leave device(s) from the platform
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Leave_{}.log &".format(LogsPath, VM['IP'])
            LeavePlatformLine="plink -t -batch -pw {} {}@{} {}LeavePlatform{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
            batfile = open(batLeavePlatform, "a")
            batfile.write(LeavePlatformLine)
            batfile.close()

    batfile = open(batMoveFiles, "a")
    batfile.write(moveFilesMkdirLine)
    batfile.write(moveFilesLine)
    if VM['OS'] == "Linux":
        batfile.write(mkExecuteFilesLine)
    batfile.close()
    # Create bat file for Prerequisite Installation for the platform
    if Setup['TestMgrOS'] == "Linux":
        ProcessTail = " > {}Prerequisite_{}.log &".format(LogsPath, VM['IP'])
    if VM['RunAsContainer']:
        IIPPrerequisiteInstallationLine="{} {} plink -t -batch -pw {} {}@{} {}DockerInstallation{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
    else:
        IIPPrerequisiteInstallationLine="{} {} plink -t -batch -pw {} {}@{} {}IIPPrerequisiteInstallation{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
    batfile = open(batIIPPrerequisiteInstallation, "a")
    batfile.write(IIPPrerequisiteInstallationLine)
    batfile.close()
    # Create bat file to run docker registry
    if VM['DockerRegistry']:
        runDockerRegistryLine="plink -t -batch -pw {} {}@{} {}RunDockerRegistry{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['DockerRegistry'], Setup['DockerRegistryPort'])
        batfile = open(batRunDockerRegistry, "a")
        batfile.write(runDockerRegistryLine)
        batfile.close()
        RegistryList.append(VM['IP'])
    else:
        runDockerRegistryLine="plink -t -batch -pw {} {}@{} {}GetDockerRegistry{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['DockerRegistry'])
        batfile = open(batRunDockerRegistry, "a")
        batfile.write(runDockerRegistryLine)
        batfile.close()

    stopDockerRegistryLine="plink -t -batch -pw {} {}@{} {}StopDockerRegistry{} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"))
    writeFile = open(batStopDockerRegistry, "a")
    writeFile.write(stopDockerRegistryLine)
    writeFile.close()
    batfile = open(batInstallPlatform, "a")
    batfile.write(installPlatformLine)
    batfile.close()
    batfile = open(batRebootVMs, "a")
    batfile.write(rebootVMLine)
    batfile.close()
    cleanAllLines="plink -t -batch -pw {} {}@{} {}CleanAll{} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"))
    batfile = open(batCleanAll, "a")
    batfile.write(cleanAllLines)
    batfile.close()

if Setup['TestMgrOS'] == "Linux":
    subprocess.run(['chmod', '-R', '755', 'Commands/Linux'])

for VM in Machines:
    
    if not VM['Included']:
        continue
        
    print("Machines in the test:")
    print(VM['Name'])
    print(VM['IP'])
    print(VM['Role'])
    print("RunAsContainer: {}".format(VM['RunAsContainer']))
    print("------------------------------------------")

Machines in the test:
Ubuntu 20.04.3 IIP Platform
192.168.81.100
Server
RunAsContainer: False
------------------------------------------
Machines in the test:
Ubuntu 20.04.3 IIP Platform Device 1
192.168.81.110
Device
RunAsContainer: False
------------------------------------------
Machines in the test:
Ubuntu 20.04.3 IIP Platform Device 2
192.168.81.120
Device
RunAsContainer: False
------------------------------------------


# Download Examples Dynamic Scripts

In [4]:
# create script to download the examples
if Setup['DownloadExamples']:
    LinuxExamplesLine = []
    PlatformURLZip="curl https://codeload.github.com/iip-ecosphere/platform/zip/refs/heads/main -o Platform.zip\n"

    LinuxExamplesLine.append(PlatformURLZip)
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("unzip Platform.zip\n")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("cp Install/src/main/easy/apps/ApplicationPartMyApp.ivml Install/src/main/easy/apps/BkApplicationPartMyApp.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("cp Install/src/main/easy/meshes/ServiceMeshPartMyAppMyMesh.ivml Install/src/main/easy/meshes/BkServiceMeshPartMyAppMyMesh.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("rm Install/src/main/easy/apps/ApplicationPart*");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("rm Install/src/main/easy/meshes/ServiceMeshPart*");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("rm Install/src/main/easy/AllServicesPart*");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("rm Install/src/main/easy/AllTypesPart*");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("mv Install/src/main/easy/apps/BkApplicationPartMyApp.ivml Install/src/main/easy/apps/ApplicationPartMyApp.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("mv Install/src/main/easy/meshes/BkServiceMeshPartMyAppMyMesh.ivml Install/src/main/easy/meshes/ServiceMeshPartMyAppMyMesh.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("sed -i '/import AllServicesPart\\*;/d' Install/src/main/easy/AllServices.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("sed -i '/import AllTypesPart\\*;/d' Install/src/main/easy/AllTypes.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S mkdir platform-main/easy");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S cp -R Install/src/main/easy/*  platform-main/easy");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm -r platform-main/easy/apps");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm -r platform-main/easy/meshes");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm platform-main/easy/AllServices.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm platform-main/easy/AllTypes.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm platform-main/easy/PlatformConfiguration.ivml");
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm platform-main/easy/TechnicalSetup.ivml");
    LinuxExamplesLine.append("\n")
    
    ExCount=0;
    for Ex in Examples:
        if not Ex['Included']:
            continue
        
        if Ex['Name'] == "SimpleMesh":
            if Ex['Container']:
                LinuxExamplesLine.append("sed -i '/Application /a \\ \\ \\ \\ \\ \\ \\ \\ createContainer = true,' Install/src/main/easy/apps/ApplicationPartMyApp.ivml");
                LinuxExamplesLine.append("\n")
                LinuxExamplesLine.append("\n")
        
        if not Ex['Redownload']:
            continue

        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("cd platform-main/platform/examples/{}".format(Ex['Name']));
        LinuxExamplesLine.append("\n")
        
        if Ex['Container']:
            LinuxExamplesLine.append("sed -i '/Application /a \\ \\ \\ \\ \\ \\ \\ \\ createContainer = true,' src/test/easy/apps/ApplicationPart*.ivml");
            LinuxExamplesLine.append("\n")
        
        LinuxExamplesLine.append("echo $1 | sudo -S find . -name \"ApplicationPart*\" -exec cp {} ../../../../Install/src/main/easy/apps/ \\;");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("echo $1 | sudo -S find . -name \"ServiceMeshPart*\" -exec cp {} ../../../../Install/src/main/easy/meshes/ \\;");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("echo $1 | sudo -S find . -name \"AllServicesPart*\" -exec cp {} ../../../../Install/src/main/easy/ \\;");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("echo $1 | sudo -S find . -name \"AllTypesPart*\" -exec cp {} ../../../../Install/src/main/easy/ \\;");
        LinuxExamplesLine.append("\n")

        if Ex['Name'] == "examples.KODEX":
            LinuxExamplesLine.append("cd {}".format(Ex['Name']));
            LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("echo $1 | sudo -S mkdir -p src/main/easy");
        LinuxExamplesLine.append("\n")
        if Ex['Name'] == "examples.KODEX":
            LinuxExamplesLine.append("echo $1 | sudo -S cp -R ../../../../easy/* src/main/easy");
        else:
            LinuxExamplesLine.append("echo $1 | sudo -S cp -R ../../../easy/* src/main/easy");
        LinuxExamplesLine.append("\n")
        
        if Ex['Name'] == "examples.rtsa":
            LinuxExamplesLine.append("echo $1 | sudo -S mkdir -p ../../../../Install/resources/rtsa/");
            LinuxExamplesLine.append("\n")
            LinuxExamplesLine.append("echo $1 | sudo -S chown -R $USER ../../../../Install/resources/rtsa/");
            LinuxExamplesLine.append("\n")
            LinuxExamplesLine.append("echo $1 | sudo -S cp -r resources/rtsa/* ../../../../Install/resources/rtsa/");
            LinuxExamplesLine.append("\n")
            
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        if Ex['Name'] == "examples.KODEX":
            LinuxExamplesLine.append("cd ..");
            LinuxExamplesLine.append("\n")
            
        ExCount += 1
    
    print(ExCount)
    if ExCount!=0:
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("sed -i '/import AllTypes; /a \\ \\ \\ \\ import AllServicesPart*;' Install/src/main/easy/AllServices.ivml");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("sed -i '/import IIPEcosphere; /a \\ \\ \\ \\ import AllTypesPart*;' Install/src/main/easy/AllTypes.ivml");
        LinuxExamplesLine.append("\n")      
    
    batLinuxExamples = "{}DownloadExamples.sh".format(AddScriptsServerPath)
    batfile = open(batLinuxExamples, "w")
    for Line in LinuxExamplesLine:
        batfile.write(Line)
    batfile.close()
    
    WinExamplesLine = []
    WinExamplesLine.append("cd Files")
    WinExamplesLine.append("\n")
    WinExamplesLine.append(PlatformURLZip)
    WinExamplesLine.append("\n")
    WinExamplesLine.append("tar xzpvf Platform.zip\n")
    WinExamplesLine.append("\n")
    WinExamplesLine.append("if not exist examples mkdir examples\n")
    WinExamplesLine.append("\n")

    for Ex in Examples:
        if not Ex['Included']:
            continue
        if not Ex['Redownload']:
            continue
            
        WinExamplesLine.append("rmdir /s examples\{} /q\n".format(Ex['Name']))
        WinExamplesLine.append("mkdir examples\{}\n".format(Ex['Name']))
        WinExamplesLine.append("Xcopy platform-main\platform\examples\{} examples\{} /E /H /C /I\n".format(Ex['Name'], Ex['Name']))
        WinExamplesLine.append("\n")

    WinExamplesLine.append("del /f Platform.zip\n")
    WinExamplesLine.append("\n")
    WinExamplesLine.append("rmdir /s platform-main /q\n")
    WinExamplesLine.append("\n")

    batWinExamples = "{}DownloadExamples.bat".format(AddScriptsServerPath)
    batfile = open(batWinExamples, "w")
    for Line in WinExamplesLine:
        batfile.write(Line)
    batfile.close()

    for Ex in Examples:
        if not Ex['Included']:
            continue
        print(Ex['Name'])
        
    print("Download Examples scripted is prepared")
else:
    batLinuxExamples = "{}DownloadExamples.sh".format(AddScriptsServerPath)
    batfile = open(batLinuxExamples, "w")
    batfile.write("")
    batfile.close()
    
    batWinExamples = "{}DownloadExamples.bat".format(AddScriptsServerPath)
    batfile = open(batWinExamples, "w")
    batfile.write("")
    batfile.close()
    
    print("Download Examples scripted is empty")

4
SimpleMesh
examples.python
examples.pythonSync
examples.rtsa
examples.KODEX
Download Examples scripted is prepared


# Compile Examples Dynamic Scripts

In [5]:
# create script to compile the examples
if Setup['CompileExamples']:
    LinuxExamplesLine = []
    
    for Ex in Examples:
        if not Ex['Included']:
            continue
        if not Ex['Redownload']:
            continue

        LinuxExamplesLine.append("cd platform-main/platform/examples/{}".format(Ex['Name']));
        LinuxExamplesLine.append("\n")
        
        if Ex['Name'] == "examples.KODEX":
            LinuxExamplesLine.append("cd {}".format(Ex['Name']));
            LinuxExamplesLine.append("\n")

        LinuxExamplesLine.append("sed -i 's/sharedArtifact = /\/\/sharedArtifact = /g' src/test/easy/TechnicalSetup.ivml");
        LinuxExamplesLine.append("\n")
        
        LinuxExamplesLine.append("sharedArtifact=$(cat src/test/easy/TechnicalSetup.ivml | grep \"sharedArtifact =\" | cut -d : -f 2)");
        LinuxExamplesLine.append("\n")
        
        LinuxExamplesLine.append("sed -i 's/'$sharedArtifact'/apps.AppInterfaces/g' pom.xml");
        LinuxExamplesLine.append("\n")
        
        LinuxExamplesLine.append("echo $1 | sudo -S mvn -P App install");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("echo \"Example {} is compiled\"".format(Ex['Name']));
        LinuxExamplesLine.append("\n")
        
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        LinuxExamplesLine.append("cd ..");
        LinuxExamplesLine.append("\n")
        if Ex['Name'] == "examples.KODEX":
            LinuxExamplesLine.append("cd ..");
            LinuxExamplesLine.append("\n")
            
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm Platform.zip\n")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S rm -r platform-main\n")
    LinuxExamplesLine.append("\n")
    
    batLinuxCompileExamples = "{}CompileExamples.sh".format(AddScriptsServerPath)
    batfile = open(batLinuxCompileExamples, "w")
    for Line in LinuxExamplesLine:
        batfile.write(Line)
    batfile.close()
    
    WinExamplesLine = []
    WinExamplesLine.append("cd Files")
    WinExamplesLine.append("\n")
    WinExamplesLine.append(PlatformURLZip)
    WinExamplesLine.append("\n")
    WinExamplesLine.append("tar xzpvf Platform.zip\n")
    WinExamplesLine.append("\n")
    WinExamplesLine.append("if not exist examples mkdir examples\n")
    WinExamplesLine.append("\n")

    for Ex in Examples:
        if not Ex['Included']:
            continue
        if not Ex['Redownload']:
            continue
            
        WinExamplesLine.append("rmdir /s examples\{} /q\n".format(Ex['Name']))
        WinExamplesLine.append("mkdir examples\{}\n".format(Ex['Name']))
        WinExamplesLine.append("Xcopy platform-main\platform\examples\{} examples\{} /E /H /C /I\n".format(Ex['Name'], Ex['Name']))
        WinExamplesLine.append("\n")

    WinExamplesLine.append("del /f Platform.zip\n")
    WinExamplesLine.append("\n")
    WinExamplesLine.append("rmdir /s platform-main /q\n")
    WinExamplesLine.append("\n")

    batWinCompileExamples = "{}CompileExamples.bat".format(AddScriptsServerPath)
    batfile = open(batWinCompileExamples, "w")
    for Line in WinExamplesLine:
        batfile.write(Line)
    batfile.close()

    for Ex in Examples:
        if not Ex['Included']:
            continue
        print(Ex['Name'])
        
    print("Compile Examples scripted is prepared")
else:
    batLinuxCompileExamples = "{}CompileExamples.sh".format(AddScriptsServerPath)
    batfile = open(batLinuxCompileExamples, "w")
    batfile.write("")
    batfile.close()
    
    batWinCompileExamples = "{}CompileExamples.bat".format(AddScriptsServerPath)
    batfile = open(batWinCompileExamples, "w")
    batfile.write("")
    batfile.close()
    
    print("Compile Examples scripted is empty")

SimpleMesh
examples.python
examples.pythonSync
examples.rtsa
examples.KODEX
Compile Examples scripted is prepared


# Move The Files

In [54]:
# Run script to move the auto scripts to the VMs 
if Setup['MoveScripts']:
    for VM in Machines:
        if not VM['Included']:
            continue
        print(VM['Name'])
        status,result = subprocess.getstatusoutput("ping -c1 -w2 " + VM['IP'])
        while status != 0:
            print(VM['IP'] + " is DOWN !, waiting...")
            status,result = subprocess.getstatusoutput("ping -c1 -w2 " + VM['IP'])
        print(VM['IP'] + " is UP !")
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batMoveFiles], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batMoveFiles], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    
    print("Scripts are moved to the machines")

Ubuntu 20.04.3 IIP Platform
192.168.81.100 is UP !
Scripts are moved to the machines


# Clean Maven and Artifacts Folder

In [55]:
# Run script to install the platform on VMs as Server and Device(s)
if Setup['CleanAll']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batCleanAll], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batCleanAll], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()

    print("Cleaning Maven and Artifacts Folder for the Server is completed")

# Install The Prerequisite for The Platform

In [56]:
# Run script to install the platform on VMs as Server and Device(s)
if Setup['InstallPrerequisite']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batIIPPrerequisiteInstallation], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the prerequisite installation on VMs")
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batIIPPrerequisiteInstallation], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the prerequisite installation on VMs")
        ls_output.communicate()

    print("The prerequisite installation is completed on all VMs")

# Run Docker Registry

In [57]:
# Run script to run docker private registry
if Setup['RunDockerRegistry']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batRunDockerRegistry], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batRunDockerRegistry], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    
    print("Docker private registry is runnung")

# Install The Platform

In [58]:
# Run script to install the platform on VMs as Server and Device(s)
if Setup['InstallPlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batInstallPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the installation on VMs")
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batInstallPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the installation on VMs")
        ls_output.communicate()

    print("The installation is completed on all VMs")

# Start The Platform

In [59]:
# Run script to start the platform Server
if Setup['StartPlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batStartPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batStartPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    time.sleep(3)
    
    for VM in Machines:
        if VM['Role'] == "Server":
            if not VM['Included']:
                continue
            PlatformReady = 0
            while PlatformReady == 0:
                with open("{}Start_{}.log".format(LogsPath, VM['IP'])) as f:
                    if 'Broker and Platform are Running... Please don\'t close it' in f.read():
                        PlatformReady = 1
                    else:
                        print("Waiting for the platform on {} to start".format(VM['IP']))
                        time.sleep(3)
            print("The platform on {} is started successfully".format(VM['IP']))

# Device(s) Join The Platform

In [60]:
# Run script to join the Device(s) to the platform
if Setup['DevicesJoinPlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batJoinPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batJoinPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(3)
    
    for VM in Machines:
        VM['ResourceID'] = ""
        if VM['Role'] == "Device":
            if not VM['Included']:
                continue

            DeviceJoin = 0
            DeviceID = ""
            while DeviceJoin == 0:
                with open("{}Join_{}.log".format(LogsPath, VM['IP'])) as f:
                    content = f.read()
                    if 'Ecs is Running... Please don\'t close it' in content:
                        DeviceJoin = 1
                    elif 'Ecs and ServiceMgr are Running... Please don\'t close it' in content:
                        DeviceJoin = 1
                    else:
                        print("Waiting for the device {} to join the platform".format(VM['IP']))
                        time.sleep(3)

            print("The device {} is joined the platform successfully".format(VM['IP'])) 

# Check Devices (Resource ID)

In [61]:
# Run script to check the Device(s) in the platform
if Setup['CheckDevices']:
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        if VM['Role'] == "Server":
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}CheckDevices.log &".format(LogsPath)
            checkDevicesLines="plink -t -batch -pw {} {}@{} {}CheckDevices{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
            writeFile = open(batCheckDevices, "a")
            writeFile.write(checkDevicesLines)
            writeFile.close()
            
            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCheckDevices], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCheckDevices], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            
    for VM in Machines:
        VM['ResourceID'] = ""
        if VM['Role'] == "Device":
            if not VM['Included']:
                continue

            with open("{}CheckDevices.log".format(LogsPath, 'r'))  as f:
                for line in f:
                    if VM['IP'] in line:
                        DeviceID = line
            DeviceIDInfo = DeviceID.split('  ')
            VM['ResourceID'] = DeviceIDInfo[0]
            print("The device {} has ResourceID: {}".format(VM['IP'], VM['ResourceID']))

# Start Service Manager Container

In [62]:
# Run script to start services manager for Device(s) with contianer Ecs
if Setup['StartServiceMgrContainer']:
    cliContainersCommandLine = "" 
    writeFile = open(batCliContainersCommand, "w")
    writeFile.write(cliContainersCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    if VMDevice['RunAsContainer']:
                        
                        ServiceMgrs = 0
                        for Ex in Examples:
                            if not Ex['Included']:
                                continue
                            if not Ex['Run']:
                                continue
                            if not Ex['Container']:
                                continue

                            if Ex['Device'] == VMDevice['DeviceID']:
                                ServiceMgrs += 1
                                ServiceMgrDescYml = Ex['ContainerDesc']

                        ServiceMgrDesc = "http://{}:4200/download/{}".format(Setup['MgtUiServerIP'], ServiceMgrDescYml)
                                                
                        if ServiceMgrs > 1:
                            print("ERROR:: Each device Run As Container can have only one example to execute")
                        else:
                        
                            ProcessTail = ""
                            cliContainersCommandLine = "" 
                            writeFile = open("{}ServiceMgrContainer_{}.log".format(LogsPath, VMDevice['IP']), "w")
                            writeFile.write(cliContainersCommandLine)
                            writeFile.close()

                            if Setup['TestMgrOS'] == "Linux":
                                ProcessTail = " > {}CliContainers_{}.log &".format(LogsPath, VMDevice['IP'])
                            cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "add", ServiceMgrDesc, ProcessTail)
                            writeFile = open(batCliContainersCommand, "a")
                            writeFile.write(cliContainersCommandLine)
                            writeFile.close()

                            if Setup['TestMgrOS'] == "Linux":
                                ls_output=subprocess.Popen(['sh', batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                                ls_output.communicate()
                            elif Setup['TestMgrOS'] == "Win":
                                ls_output=subprocess.Popen([batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                                ls_output.communicate()

                            cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "start", ServiceMgrDesc, ProcessTail)
                            writeFile = open(batCliContainersCommand, "w")
                            writeFile.write(cliContainersCommandLine)
                            writeFile.close()

                            if Setup['TestMgrOS'] == "Linux":
                                ProcessTail = " > {}ServiceMgrContainer_{}.log &".format(LogsPath, VMDevice['IP'])
                            cliContainersCommandLine="{} {} plink -t -batch -pw {} {}@{} {}WaitServiceMgrContainer{} {} {}\n".format(startCommand, ProcessTitle, VMDevice['Password'], VMDevice['Username'], VMDevice['IP'], VMHomeFilesPath, VMFileExt, ServiceMgrDesc, ProcessTail)
                            writeFile = open(batCliContainersCommand, "a")
                            writeFile.write(cliContainersCommandLine)
                            writeFile.close()

                            if Setup['TestMgrOS'] == "Linux":
                                ls_output=subprocess.Popen(['sh', batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                            elif Setup['TestMgrOS'] == "Win":
                                ls_output=subprocess.Popen([batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                            time.sleep(5)

                            DeviceJoin = 0
                            while DeviceJoin == 0:
                                with open("{}ServiceMgrContainer_{}.log".format(LogsPath, VMDevice['IP'])) as f:
                                    content = f.read()
                                    if 'ServiceMgr is Running... Please don\'t close it' in content:
                                        DeviceJoin = 1
                                    else:
                                        print("Waiting for the ServiceMgr Container on {}".format(VMDevice['IP']))
                                        time.sleep(3)

                            print("The ServiceMgr Container on {} is running successfully".format(VM['IP']))

# Add Services

In [63]:
# Run script to add new jar application to Device(s) in the platform
if Setup['AddServiceExamples']:

    examplesMessages = []
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()

    for VM in Machines:
        if not VM['Included']:
            continue

        if VM['Role'] == "Server":

            for Ex in Examples:
                if not Ex['Included']:
                    continue
                if not Ex['Run']:
                    continue

                ResourceID=""
                for VMDevice in Machines:
                    if not VMDevice['Included']:
                        continue
                    if VMDevice['Role'] != "Device":
                        continue

                    if VMDevice['DeviceID'] == Ex['Device']:
                        ResourceID = VMDevice['ResourceID']

                if ResourceID=="":
                    print("{} example application is NOT added to the platform - Missing resources".format(Ex['Name']))
                    continue

                ServiceURL = "http://{}:4200/download/{}".format(Setup['MgtUiServerIP'], Ex['Artifact'])

                if Setup['TestMgrOS'] == "Linux":
                    ProcessTail = " > {}AddService_{}_{}.log &".format(LogsPath, Ex['Device'], ResourceID)
                cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, ResourceID, "add", ServiceURL, ProcessTail)
                writeFile = open(batCliServicesCommand, "a")
                writeFile.write(cliServicesCommandLine)
                writeFile.close()

                examplesMessages.append("{} example application is added to the platform".format(Ex['Name']))

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()

            for Line in examplesMessages:
                print(Line)

# StartAll Services

In [64]:
# Run script to start all services in application in the platform
if Setup['StartAllServiceExamples']:
    
    examplesMessages = []
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()

    for VM in Machines:
        if not VM['Included']:
            continue

        if VM['Role'] == "Server":

            for Ex in Examples:
                if not Ex['Included']:
                    continue
                if not Ex['Run']:
                    continue

                ResourceID=""
                for VMDevice in Machines:
                    if not VMDevice['Included']:
                        continue
                    if VMDevice['Role'] != "Device":
                        continue

                    if VMDevice['DeviceID'] == Ex['Device']:
                        ResourceID = VMDevice['ResourceID']

                if ResourceID=="":
                    continue

                ArtifactDeployment = "{}/{}".format(Setup['ArtifactsFolder'], Ex['Artifact'])

                if Setup['TestMgrOS'] == "Linux":
                    ProcessTail = " > {}StartAllService_{}_{}.log &".format(LogsPath, Ex['Device'], ResourceID)
                cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, ResourceID, "startAll", ArtifactDeployment, ProcessTail)
                writeFile = open(batCliServicesCommand, "a")
                writeFile.write(cliServicesCommandLine)
                writeFile.close()

                examplesMessages.append("{} example application is started".format(Ex['Name']))

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()

            for Line in examplesMessages:
                print(Line)

    time.sleep(20)

# StopAll Services

In [65]:
# Run script to stop all services in application in the platform
if Setup['StopAllServiceExamples']:
    
    examplesMessages = []
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()

    for VM in Machines:
        if not VM['Included']:
            continue

        if VM['Role'] == "Server":

            for Ex in Examples:
                if not Ex['Included']:
                    continue
                if not Ex['Run']:
                    continue

                ResourceID=""
                for VMDevice in Machines:
                    if not VMDevice['Included']:
                        continue
                    if VMDevice['Role'] != "Device":
                        continue

                    if VMDevice['DeviceID'] == Ex['Device']:
                        ResourceID = VMDevice['ResourceID']

                if ResourceID=="":
                    continue

                ArtifactDeployment = "{}/{}".format(Setup['ArtifactsFolder'], Ex['Artifact'])

                if Setup['TestMgrOS'] == "Linux":
                    ProcessTail = " > {}StopAllService_{}_{}.log &".format(LogsPath, Ex['Device'], ResourceID)
                cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, ResourceID, "stopAll", ArtifactDeployment, ProcessTail)
                writeFile = open(batCliServicesCommand, "a")
                writeFile.write(cliServicesCommandLine)
                writeFile.close()

                examplesMessages.append("{} example application is stopped".format(Ex['Name']))

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()

            for Line in examplesMessages:
                print(Line)


# Remove Services

In [66]:
# Run script to remove application from Device(s) in the platform
if Setup['RemoveServiceExamples']:
        
    examplesMessages = []
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()

    for VM in Machines:
        if not VM['Included']:
            continue

        if VM['Role'] == "Server":

            for Ex in Examples:
                if not Ex['Included']:
                    continue
                if not Ex['Run']:
                    continue

                ResourceID=""
                for VMDevice in Machines:
                    if not VMDevice['Included']:
                        continue
                    if VMDevice['Role'] != "Device":
                        continue

                    if VMDevice['DeviceID'] == Ex['Device']:
                        ResourceID = VMDevice['ResourceID']

                if ResourceID=="":
                    continue

                ArtifactDeployment = "{}/{}".format(Setup['ArtifactsFolder'], Ex['Artifact'])

                if Setup['TestMgrOS'] == "Linux":
                    ProcessTail = " > {}RemoveService_{}_{}.log &".format(LogsPath, Ex['Device'], ResourceID)
                cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, ResourceID, "remove", ArtifactDeployment, ProcessTail)
                writeFile = open(batCliServicesCommand, "a")
                writeFile.write(cliServicesCommandLine)
                writeFile.close()

                examplesMessages.append("{} example application is removed".format(Ex['Name']))

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()

            for Line in examplesMessages:
                print(Line)


# Deploy a deployment plan

In [67]:
# Deploy a deployment plan in the platform
if Setup['RunDeploymentPlans']:

    cliDeployCommandLine = "" 
    writeFile = open(batCliDeployCommand, "w")
    writeFile.write(cliDeployCommandLine)
    writeFile.close()

    for VM in Machines:
        if not VM['Included']:
            continue

        if VM['Role'] == "Server":

            for Deploy in Deployments:
                if not Deploy['Run']:
                    continue
                DeploymentURL = "http://{}:4200/download/{}".format(Setup['MgtUiServerIP'], Deploy['yaml'])

                if Setup['TestMgrOS'] == "Linux":
                    ProcessTail = " > {}deploy_{}.log &".format(LogsPath, Deploy['Name'])
                cliDeployCommandLine="plink -t -batch -pw {} {}@{} {}CliDeployCommand{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, "deploy", DeploymentURL, ProcessTail)
                writeFile = open(batCliDeployCommand, "a")
                writeFile.write(cliDeployCommandLine)
                writeFile.close()

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliDeployCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliDeployCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()

            for Deploy in Deployments:
                if not Deploy['Run']:
                    continue
                print("{} deployment is added to the platform".format(Deploy['Name']))

            time.sleep(20)

# Undeploy a deployment plan

In [68]:
# Undeploy a deployment plan in the platform
if Setup['StopDeploymentPlans']:

    cliDeployCommandLine = "" 
    writeFile = open(batCliDeployCommand, "w")
    writeFile.write(cliDeployCommandLine)
    writeFile.close()

    for VM in Machines:
        if not VM['Included']:
            continue

        if VM['Role'] == "Server":

            for Deploy in Deployments:
                if not Deploy['Run']:
                    continue
                DeploymentURL = "http://{}:4200/download/{}".format(Setup['MgtUiServerIP'], Deploy['yaml'])

                if Setup['TestMgrOS'] == "Linux":
                    ProcessTail = " > {}undeploy_{}.log &".format(LogsPath, Deploy['Name'])
                cliDeployCommandLine="plink -t -batch -pw {} {}@{} {}CliDeployCommand{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, "undeploy", DeploymentURL, ProcessTail)
                writeFile = open(batCliDeployCommand, "a")
                writeFile.write(cliDeployCommandLine)
                writeFile.close()

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliDeployCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliDeployCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()

            for Deploy in Deployments:
                if not Deploy['Run']:
                    continue
                print("{} deployment is removed to the platform".format(Deploy['Name']))

# Stop Service Manager Container

In [69]:
# Run script to stop services manager for Device(s) with contianer Ecs
if Setup['StopServiceMgrContainer']:
    cliContainersCommandLine = "" #plink -t -batch -pw {} {}@{} \"chmod 755 {}Files/*{}\"\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomePath, VMFileExt)
    writeFile = open(batCliContainersCommand, "w")
    writeFile.write(cliContainersCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        ServiceMgrDesc = "http://{}:4200/download/SimplePythondemoflowapp_dflt_mesh-info.yml".format(Setup['MgtUiServerIP'])
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    if VMDevice['RunAsContainer']:
                        
                        ServiceMgrs = 0
                        for Ex in Examples:
                            if not Ex['Included']:
                                continue
                            if not Ex['Run']:
                                continue
                            if not Ex['Container']:
                                continue

                            if Ex['Device'] == VMDevice['DeviceID']:
                                ServiceMgrs += 1
                                ServiceMgrDescYml = Ex['ContainerDesc']

                        ServiceMgrDesc = "http://{}:4200/download/{}".format(Setup['MgtUiServerIP'], ServiceMgrDescYml)
                                                
                        if ServiceMgrs > 1:
                            print("ERROR:: Each device Run As Container can have only one example to execute")
                        else:

                            cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "stop", ServiceMgrDesc)
                            writeFile = open(batCliContainersCommand, "a")
                            writeFile.write(cliContainersCommandLine)
                            writeFile.close()
                            # Create bat file to stop Ecs container on the Device(s)
                            cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "undeploy", ServiceMgrDesc)
                            writeFile = open(batCliContainersCommand, "a")
                            writeFile.write(cliContainersCommandLine)
                            writeFile.close()

                            if Setup['TestMgrOS'] == "Linux":
                                ls_output=subprocess.Popen(['sh', batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                                ls_output.communicate()
                            elif Setup['TestMgrOS'] == "Win":
                                ls_output=subprocess.Popen([batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                                ls_output.communicate()
                                
                            print("The ServiceMgr Container on {} is stopped and removed successfully".format(VM['IP']))


# Device(s) Leave The Platform

In [70]:
# Run script to stop and leave the Device(s) from the platform
if Setup['DevicesLeavePlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batLeavePlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batLeavePlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
        
    print("The device(s) left the platform successfully")

# Stop The Platform

In [71]:
# Run script to stop the platform Server
if Setup['StopPlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batStopPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batStopPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    
    print("The platform is stopped successfully")

# Stop Docker Registry

In [72]:
# Run script to run docker private registry
if Setup['StopDockerRegistry']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batStopDockerRegistry], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batStopDockerRegistry], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    
    print("Docker private registry is stopped and removed")

Docker private registry is stopped and removed
